In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from torch import optim
import matplotlib.pyplot as plt

In [2]:
torch.manual_seed(110)
x1= torch.randn((3,5), dtype=torch.float32)
w1= torch.randn((5,), dtype=torch.float32)

x= torch.tensor([[1. ,2. ,0. ,4. ,1.],
                 [0. ,1. ,1. ,1. ,1.],
                 [2. ,3. ,0. ,1. ,4.]])
w= torch.tensor([1., 0.5, 1., -1, -0.5])
b= torch.tensor(1.0)

In [3]:
x1

tensor([[-0.4542, -1.1170,  0.7534,  2.6707, -1.2473],
        [-0.8668,  0.1647, -0.1978, -1.1574, -0.0619],
        [-1.8319, -0.4143, -0.4898,  1.4640, -1.8555]])

In [4]:
w1

tensor([ 0.2380,  0.8970, -1.8386,  2.0783, -0.5478])

🔶activation function

In [5]:
def activation_linear(x):
    return x

🔶step function

In [6]:
def activation_step(x):
    if x>0:
        y= torch.tensor(1.)
    elif x<0:
        y=torch.tensor(0.)
    else:
        y=torch.tensor(0.5)
    return y
        

🔶neurun function

In [7]:
def neuron(x, w, b , af):
    z=0
    for xi,wi in zip(x,w):
        z += xi*wi
    z += b
    y = af(z)
    return y

In [8]:
print(x1[0])
print(x[0])

tensor([-0.4542, -1.1170,  0.7534,  2.6707, -1.2473])
tensor([1., 2., 0., 4., 1.])


In [9]:
neuron(x1[0], w1 , b , activation_linear)

tensor(4.7384)

In [10]:
neuron(x[0], w , b , activation_step) 

tensor(0.)

In [11]:
#(input , value)تابع پله
torch.heaviside(torch.tensor(-1.),torch.tensor(0.5))

tensor(0.)

🔶vectorization

In [12]:
def neuron2(x , w , b ,af):
    #y= af(torch.matmul(x , w))
    y= af(x @ w + b)
    return y 

In [13]:
neuron(x[0], w , b , activation_step) 

tensor(0.)

🔶class neuron 

In [14]:
class Neuron:
    def __init__(self, m , af):
        self.w = torch.randn(m)
        self.b = torch.randn(1)
        self.af = af
    def __call__(self,x):
        y = self.af(x @ self.w + b)
        return y
    

In [15]:
neuron3 = Neuron(5, activation_linear)

In [16]:
neuron3(x[0])

tensor(-2.5586)

✳️💠LOSS✅

In [17]:
yp = torch.randn((10,), dtype = torch.float32)
yt = torch.randn((10,), dtype = torch.float32)
yp

tensor([-0.3337,  0.4987,  0.1842, -0.6225,  0.8596,  0.5227, -0.0219, -0.3622,
        -0.9103, -0.4838])

💠from scratch

In [18]:
torch.mean((yp - yt)**2)

tensor(1.9661)

💠MSE in pytorch

In [19]:
#از این ادرس بصورت کلاس باید ازش استفاده کنیم
mse_loss = nn.MSELoss()
mse_loss(yp , yt)

tensor(1.9661)

💠MSE function

In [20]:
#از این ادرس بصورت فانکشی در دسترس هست
F.mse_loss(yp , yt)

tensor(1.9661)

💠MAS loss in pytorch

In [21]:
MAE_loss = nn.L1Loss()
MAE_loss(yp , yt)

tensor(1.1682)

💠MAE function

In [22]:
F.l1_loss(yt , yp)

tensor(1.1682)

💠 connect loss to neuron

In [23]:
class Neuron:
    def __init__(self, m , af):
        torch.manual_seed(110)
        self.w = torch.randn(m)
        self.b = torch.randn(1)
        self.af = af
    def __call__(self , x):
        y = x @ self.w + self.b
        if self.af == 'linear':
            y = self.linear(y)
        elif self.af=='step':
            y = self.step(y)
        return y
    def linear(self , a):
        return a
    def step(self , b):
        if b>0 :
            y= torch.tensor(1.)
        elif b<0:
            y=torch.tensor(0.)
        else:
            y=torch.tensor(0.5)
        return y

In [24]:
x=torch.randn((3,5), dtype = torch.float32)
yt= torch.randn((3,), dtype = torch.float32)

x2= torch.tensor([[1. ,2. ,0. ,4. ,1.],
                 [0. ,1. ,1. ,1. ,1.],
                 [2. ,3. ,0. ,1. ,4.]])
yt2= torch.tensor([1., 2, 0.])

neuron4 = Neuron(5 , 'linear')
yp = neuron4(x2[0])

print(yt2[0:1])
print(yp)

loss= F.mse_loss(yp,yt2[0:1])
loss
 

tensor([1.])
tensor([5.8804])


tensor(23.8184)

✏️〽️autogradad ✏️

In [25]:
x = torch.tensor(3. , requires_grad=True)
y=x**2
y.backward()
print(x.grad)
x.grad.zero_()

tensor(6.)


E:\workplace\python_install\Lib\site-packages\torch\autograd\graph.py:825: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 9020). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\cuda\CUDAFunctions.cpp:108.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


tensor(0.)

〽️connect gradfunction to training block diagram

In [26]:
x5= torch.tensor([[1. ,2. ,0. ,4. ,1.],
                 [0. ,1. ,1. ,1. ,1.],
                 [2. ,3. ,0. ,1. ,4.]])
yt5= torch.tensor([1., 2.0, 0.])

In [27]:
class Neuron5:
    def __init__(self, m , af):
        torch.manual_seed(110)
        self.w = torch.randn(m , requires_grad=True)
        self.b = torch.randn(1 , requires_grad=True)
        self.af = af
    def __call__(self , x):
        y = x @ self.w + self.b
        if self.af == 'linear':
            y = self.linear(y)
        elif self.af=='step':
            y = self.step(y)
        return y
    def linear(self , a):
        return a
    def step(self , b):
        pass

In [28]:
neuron5 = Neuron5(5 , 'linear')
print(neuron5.w)
print(neuron5.b)
yp5 = neuron5(x5[0])

print(yp5)

tensor([-0.4542, -1.1170,  0.7534,  2.6707, -1.2473], requires_grad=True)
tensor([-0.8668], requires_grad=True)
tensor([5.8804], grad_fn=<AddBackward0>)


In [29]:
loss5= F.mse_loss(yp5,yt5[0:1])
loss5

tensor(23.8184, grad_fn=<MseLossBackward0>)

In [30]:
loss5.backward()

In [31]:
neuron5.w.grad

tensor([ 9.7608, 19.5216,  0.0000, 39.0433,  9.7608])

In [32]:
neuron5.b.grad

tensor([9.7608])

🟡💠optimizers🟡

In [33]:
def funcx2(x):
    return x**2

In [34]:
xi= torch.tensor(-3. , requires_grad = True)

In [35]:
params= [xi]
eta= 0.1
optimizer = optim.SGD(params , eta )
optimizer

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 0.1
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)

In [36]:
N = 30
for i in range (N):
    funcx2(xi).backward()
    """xi -= xi*xi.grad
    xi.grad.zero_()"""
    optimizer.step()
    optimizer.zero_grad()
    

In [37]:
xi

tensor(-0.0037, requires_grad=True)

In [38]:
optimizer.param_groups

[{'params': [tensor(-0.0037, requires_grad=True)],
  'lr': 0.1,
  'momentum': 0,
  'dampening': 0,
  'weight_decay': 0,
  'nesterov': False,
  'maximize': False,
  'foreach': None,
  'differentiable': False,
  'fused': None}]

In [39]:
optimizer.param_groups[0]['params']

[tensor(-0.0037, requires_grad=True)]

🟡connect optimizer to neuron

In [40]:
x6 = torch.tensor([[1. ,2. ,0. ,4. ,1.],
                 [0. ,1. ,1. ,1. ,1.],
                 [2. ,3. ,0. ,1. ,4.]])
yt6 = torch.tensor([1., 2.0, 0.])

neuron6 = Neuron5(5 , 'linear')
params = [neuron6.w , neuron6.b]
eta = 0.1
N = 5
optimizer = optim.SGD(params, eta)

for i in range(N):
    print(x6[0])
    yp6 = neuron6(x6[0])
    loss = F.mse_loss(yp6 ,yt6[0])
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    



tensor([1., 2., 0., 4., 1.])
tensor([1., 2., 0., 4., 1.])
tensor([1., 2., 0., 4., 1.])
tensor([1., 2., 0., 4., 1.])
tensor([1., 2., 0., 4., 1.])


C:\Users\Azadeh\AppData\Local\Temp\ipykernel_3856\2608549189.py:15: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(yp6 ,yt6[0])


🟡fully connected neurons

In [41]:
fc = nn.Linear(in_features=4 , out_features=5 , bias=False)
fc.bias

In [42]:
fc.weight

Parameter containing:
tensor([[-0.0174,  0.4588, -0.3709,  0.0870],
        [-0.1864,  0.2080, -0.4389,  0.2027],
        [-0.3081, -0.3095, -0.1464, -0.4760],
        [ 0.1627,  0.0315,  0.0870, -0.1559],
        [ 0.1850,  0.3201, -0.3967, -0.2373]], requires_grad=True)

In [43]:
x = torch.randn(6,4)
x

tensor([[-0.9658,  0.8763, -0.0505,  0.8467],
        [ 1.2977,  0.6857,  0.5270, -0.9731],
        [-0.3251, -0.0119,  1.5871,  0.6515],
        [ 1.9699, -0.2955, -0.8002, -0.8998],
        [-1.5724, -0.0874, -1.4357, -0.8746],
        [-0.6784, -0.1275, -0.2685, -1.5063]])

In [44]:
fc(x) 

tensor([[ 0.5113,  0.5562, -0.3693, -0.2659, -0.0790],
        [ 0.0118, -0.5278, -0.2260,  0.4303,  0.4814],
        [-0.5318, -0.5063, -0.4386, -0.0167, -0.8482],
        [ 0.0485, -0.2600,  0.0300,  0.3818,  0.8008],
        [ 0.4438,  0.7278,  1.1380, -0.2471,  0.4583],
        [-0.0781, -0.0876,  1.0047,  0.0971,  0.2977]], grad_fn=<MmBackward0>)

🟡multi layer

In [45]:
x = torch.randn(6,3)
fc1= nn.Linear(3,8)
fc2= nn.Linear(8,5 , bias=False)
fc2(fc1(x))

tensor([[-0.1887,  0.2825,  0.5871,  0.4770, -0.3234],
        [ 0.0812, -0.2457,  0.6644,  0.4276,  0.0043],
        [-0.1599, -0.3534,  0.4591,  0.4640,  0.3018],
        [ 0.0198,  0.0974,  0.4914,  0.3080, -0.2550],
        [-0.0326,  0.3447,  0.4112,  0.2556, -0.4458],
        [-0.0268, -0.0929, -0.6707, -0.3847,  0.1955]], grad_fn=<MmBackward0>)

In [46]:
mlp = nn.Sequential(
    nn.Linear(3,8),
    nn.Linear(8,5)
)

mlp(x)

tensor([[-0.6512,  0.0062, -0.2367, -0.1428,  0.0196],
        [-0.7027,  0.1253, -0.0912,  0.0486, -0.1218],
        [-1.0618,  0.1768,  0.3217,  0.0687, -0.1629],
        [-0.5063, -0.0133, -0.2813,  0.0027,  0.0770],
        [-0.3814, -0.1000, -0.4204, -0.0482,  0.1950],
        [-0.5437, -0.2215,  0.2267,  0.4331,  0.5524]],
       grad_fn=<AddmmBackward0>)

In [47]:
#having access to layers
mlp[0].weight

Parameter containing:
tensor([[-0.4902,  0.0838,  0.0706],
        [-0.3610, -0.0157, -0.3017],
        [-0.4947, -0.0050, -0.3457],
        [ 0.1358, -0.3073,  0.5544],
        [-0.3446, -0.1883, -0.2568],
        [-0.0406,  0.2430,  0.4883],
        [ 0.1049,  0.0137,  0.4587],
        [-0.1429,  0.1643,  0.0624]], requires_grad=True)

🟡implementing a mlp neural   

In [99]:
x_train = torch.randn(80, dtype=torch.float32)
y_train = torch.randn(80, dtype=torch.float32)

In [100]:
#x should be two dimentionl but y is ok
x_train = x_train.unsqueeze(1)
print(x_train)
print(x_train.shape)

tensor([[ 1.8447],
        [-0.4474],
        [-0.4835],
        [-0.9534],
        [-0.3841],
        [ 0.1763],
        [-0.4426],
        [-0.6049],
        [ 0.7302],
        [-1.4584],
        [-0.1647],
        [ 0.2698],
        [ 0.7953],
        [-0.5099],
        [-0.7832],
        [ 0.1800],
        [-0.1460],
        [ 1.5791],
        [ 0.0481],
        [ 0.2790],
        [-0.2691],
        [-1.3943],
        [ 0.6794],
        [ 0.3442],
        [-0.4396],
        [-0.6744],
        [ 0.1463],
        [ 1.0985],
        [-1.1072],
        [-0.3778],
        [-0.1853],
        [-0.3606],
        [-0.2616],
        [ 1.2556],
        [ 0.5531],
        [-1.3186],
        [-0.7989],
        [-0.0451],
        [ 2.1893],
        [ 1.3173],
        [ 1.0590],
        [ 1.1387],
        [ 0.3749],
        [-0.9765],
        [ 0.3862],
        [ 1.0982],
        [-1.3427],
        [-1.4197],
        [-1.0386],
        [-0.5594],
        [ 0.8807],
        [ 0.8817],
        [-1.

In [101]:
y_train

tensor([ 1.1416,  1.5690, -0.8040,  0.8074, -0.2738,  0.0626, -1.1962,  0.0390,
        -0.2206, -0.1446, -0.2945,  0.4354, -0.6426,  2.0830,  1.1026,  0.2785,
         1.1067,  0.2534, -1.1139,  0.6588,  0.8038,  0.6883,  0.7408, -0.5847,
         0.7056, -1.2366, -0.3231, -0.3726,  0.6833, -1.0665, -0.5256, -0.4433,
        -1.5699,  0.1904,  0.2411, -0.0284,  0.3951,  0.3430,  0.9633,  0.9146,
        -2.1405, -0.0470,  0.0228, -0.5431, -0.0338,  0.2256, -0.9730,  0.6923,
        -0.4048,  1.7726, -0.9225, -0.1678,  0.6603,  1.2264,  0.3876,  0.2045,
         1.3522,  1.1530, -2.0876,  0.4680,  2.5461, -1.3754,  0.8489,  0.4856,
         0.6433,  0.4062,  1.0413, -1.5189,  3.5079,  0.0287,  1.4705,  1.0622,
         1.7695, -0.3275,  2.0450,  0.7422, -0.1752, -0.9510, -1.8318, -0.5849])

In [102]:
#plt.scatter(x_train , y_train)

In [158]:
model = nn.Sequential(
    nn.Linear(1,3),
    nn.Linear(3,1)
)
model

Sequential(
  (0): Linear(in_features=1, out_features=3, bias=True)
  (1): Linear(in_features=3, out_features=1, bias=True)
)

In [159]:
#loss_func = nn.MSELoss()
loss_func= nn.L1Loss()

In [160]:
for name , param in model.named_parameters():
    print(name , param)

0.weight Parameter containing:
tensor([[ 0.3492],
        [-0.2902],
        [-0.2293]], requires_grad=True)
0.bias Parameter containing:
tensor([-0.7732, -0.6355, -0.7027], requires_grad=True)
1.weight Parameter containing:
tensor([[ 0.5125, -0.1113, -0.4003]], requires_grad=True)
1.bias Parameter containing:
tensor([0.1993], requires_grad=True)


In [161]:
lr=0.5
optimizer = optim.SGD(model.parameters() , lr)

In [162]:
N = 50
loss_list=[]
for iter in range(N):
    yp = model(x_train)
    #print(yp.squeeze().shape)
    #print(y_train.shape)
    loss = loss_func(yp.squeeze() , y_train)
    loss_list.append(loss)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    print(loss.item())

0.8642706871032715
0.8359402418136597
0.82757967710495
0.8206332325935364
0.8167308568954468
0.813133716583252
0.8112398982048035
0.808444619178772
0.8060411214828491
0.8064254522323608
0.8046520352363586
0.8038781881332397
0.8041669726371765
0.8000258207321167
0.8003033399581909
0.7988676428794861
0.7998695969581604
0.7971790432929993
0.7980247735977173
0.7986788749694824
0.8012651205062866
0.7978143692016602
0.7985727190971375
0.7994977831840515
0.7969576716423035
0.798551082611084
0.7995057106018066
0.7969600558280945
0.7985295057296753
0.7995136380195618
0.7969624400138855
0.7985081076622009
0.7995214462280273
0.796964704990387
0.7984868288040161
0.7995292544364929
0.7969670295715332
0.7984656095504761
0.7995370030403137
0.7969693541526794
0.798444390296936
0.7995446920394897
0.7969716191291809
0.7984232902526855
0.799552321434021
0.7969738245010376
0.7984023690223694
0.7995599508285522
0.7969760298728943
0.7983814477920532


In [97]:
#loss_list